In [48]:

# %%
import os
import glob
import copy
import sys
import pickle 
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np 


from rslds_visualization_utils import plot_states_and_neurons, plot_states_and_neurons_state_list
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap

sys.path.append("/Users/friederikebuck/Desktop/MBL/project/WholeBrainImagingAnalysis/collab/")
sys.path.append('/Users/bennetsakelaris/Documents/Obsidian Vault/Worms/wormcode/Code+Notes 09-24-24/collab/')

sys.path.append("/Users/friederikebuck/Downloads/worm notes/worm_code/")


from get_data.read_neuro_beh_data import get_exp_features, load_json
from get_data.process_neural_data import get_derivative_of_neural_activity, get_neural_activity_from_labels, get_neural_activity_and_labels
from beh_classification.behavior_features import get_behavior_features
from get_data.read_neuro_beh_data import get_exp_features, load_json

sys.path.append('/Users/friederikebuck/Desktop/MBL/project/WholeBrainImagingAnalysis/collab/')
from beh_classification.process_midlines.midline_utils import save_as_csv, load_from_csv, convert_coordinates_to_angles
from rslds_motor_state_class_fncs import get_motor_state_start_end_is_rslds_start_end_specific

from load_data_fncs import load_all_data, get_exp_dates


def find_runs(x):
    """Find start indices, end indices, and values of runs in a 1D array."""
    n = len(x)
    if n == 0:
        return np.array([], dtype=int), np.array([], dtype=int), np.array([], dtype=x.dtype)

    change_idx = np.diff(x, prepend=x[0]-1).nonzero()[0]
    start_idx = change_idx
    end_idx = np.append(change_idx[1:], n)
    values = x[start_idx]
    return start_idx, end_idx, values


def make_masks(labels):
    labels = np.asarray(labels)
    start_idx, end_idx, values = find_runs(labels)

    mask_1_to_2 = np.zeros_like(labels, dtype=int)
    mask_1_to_0 = np.zeros_like(labels, dtype=int)
    mask_2_after_1 = np.zeros_like(labels, dtype=int)
    mask_2_after_0 = np.zeros_like(labels, dtype=int)

    for i in range(len(values)):
        val = values[i]
        s, e = start_idx[i], end_idx[i]

        if val == 1:
            if i + 1 < len(values) and values[i + 1] == 2:
                mask_1_to_2[s:e] = 1
            elif i + 1 < len(values) and values[i + 1] == 0:
                mask_1_to_0[s:e] = 1

        elif val == 2:
            if i - 1 >= 0 and values[i - 1] == 1:
                mask_2_after_1[s:e] = 1
            elif i - 1 >= 0 and values[i - 1] == 0:
                mask_2_after_0[s:e] = 1
                
    # ev_of_rev_turn_bin, pure_rev_bin, turn_of_rev_turn_bin, pure_turn_bin = make_masks(z[i])

    return mask_1_to_2, mask_1_to_0, mask_2_after_1, mask_2_after_0

def get_reor_mat(z):
    all_reors = []
    for z_w in z:
        rev_of_rev_turn_bin, pure_rev_bin, turn_of_rev_turn_bin, pure_turn_bin = make_masks(z_w)
        # fig, axs = plt.subplots(5,1, figsize = (24,15))
        # for i, (states, cmap, vmax) in enumerate(zip([z_w, rev_of_rev_turn_bin, pure_rev_bin, turn_of_rev_turn_bin, pure_turn_bin],
        #                                        [beh_cmap,state_cmap, state_cmap,state_cmap, state_cmap ], 
        #                                        [len(palette1), 2,2,2,2]
        #                                        )):
        #     axs[i].imshow(states[:, None].T, cmap = cmap, vmin =0 , vmax =vmax)
        #     axs[i].set_aspect('auto')
            
        
        reor_types = np.zeros(1599)
        reor_types[pure_rev_bin.astype('bool')] = 3
        reor_types[rev_of_rev_turn_bin.astype('bool')] = 4
        reor_types[turn_of_rev_turn_bin.astype('bool')] = 5
        reor_types[pure_turn_bin.astype('bool')] = 6
        all_reors.append(reor_types[:,None])
    return np.concatenate(all_reors, axis =1 ).T

    

In [4]:


def get_zero_crossing_indices(arr):
    arr = np.asarray(arr)
    signs = np.sign(arr)
    # Replace zeros with previous nonzero sign to avoid false crossings
    for i in range(1, len(signs)):
        if signs[i] == 0:
            signs[i] = signs[i-1]

    crossings = np.where(np.diff(signs) != 0)[0]
    return crossings  # indices *before* crossing


def count_zero_crossing(body_angle, starts, ends):
    total_crossings = []
    crossings_is = []
    if len(starts)==0:
        return [], np.array([])
    for start, end in zip(starts, ends):
        crossings_i = get_zero_crossing_indices(body_angle[start:end])
        crossings_is.append(crossings_i+start)
        total_crossings.append(len(crossings_i))
        
    # print(total_crossings,crossings_is )
    return total_crossings, np.concatenate(crossings_is)


In [43]:


T = 1599

motor_state_num_to_name = {
    0:"fwd", 
    1:"rev", 
    2:"turn", 
    3:"pure rev", 
    4:"rev of rev turn", 
    5:"turn of rev turn", 
    6:"pure turn"
}
rslds_state_num_to_name = {
    0:"fwd-like", 
    1:"rev-like", 
    2:"turn-like", 
    3:"pure rev", 
    4:"rev of rev turn", 
    5:"turn of rev turn", 
    6:"pure turn"
}

rslds_state_to_color = {
    0: "salmon", 
    1: "dodgerblue", 
    2: "green"
}


comp = "mac"
if comp == "hpc":
  project_dir = "/ru-auth/local/home/fbuck/scratch/WholeBrainImaging/keypt_moseq/projects/_1/"#sys.argv[1]+"_1"#/Volumes/FB1/moseq_project_dir/WholeBrainImagingAnalysis_1/"
  video_dir = "/ru-auth/local/home/fbuck/scratch/WholeBrainImaging/outputs_hpc/" #"/Users/friederikebuck/Desktop/MBL/project/outputs/"  #when al subdirs with vids or keypoints are stored 
  video_clip_dir = "/ru-auth/local/home/fbuck/scratch/WholeBrainImaging/outputs_hpc/vid_clips/"

else: 
#   project_dir = "/Volumes/FB1/moseq_project_dir_and_outputs/_1/"
# moseq_project_dir = ""
    project_dir = "/Volumes/FB1/moseq_project_dir_and_outputs/_1/"
    project_dir = "/Volumes/FB_data_ana/moseq_project_dir_and_outputs/_1/"
    video_dir = "/Users/friederikebuck/Desktop/MBL/project/outputs/" #"/Users/friederikebuck/Desktop/MBL/project/outputs/"  #when al subdirs with vids or keypoints are stored 
    video_clip_dir = "/Users/friederikebuck/Desktop/MBL/project/vid_clips/"
    save_dir = f"/Volumes/FB_data_ana/moseq_project_dir_and_outputs_figs/"

    # os.makedirs(save_dir, exist_ok=True)

json_dir = "/Users/friederikebuck/Desktop/MBL/project/data/Neuropal_no_heat/"
h5_dir ="/Users/friederikebuck/Desktop/MBL/project/data/processed_h5/"
beh_data_dir_parent = "/Users/friederikebuck/Desktop/MBL/project/outputs/"
        
full_traces, full_neural_labels, full_beh_classification, full_beh_data = load_all_data()
velocity_og = np.array([full_beh_data[i]["velocity"][0:1599] for i in range(len(full_beh_data))])
acceleration_og = np.array([full_beh_data[i]["acceleration"][1:1600] for i in range(len(full_beh_data))])
head_curvature_og = np.array([full_beh_data[i]["head_angle"][0:1599] for i in range(len(full_beh_data))])
worm_curvature_og = np.array([full_beh_data[i]["worm_curvature"][0:1599] for i in range(len(full_beh_data))])
pumping_og = np.array([full_beh_data[i]["pumping"][0:1599] for i in range(len(full_beh_data))])
body_angle_mag_og = np.array([full_beh_data[i]["body_angle_mag"][0:1599] for i in range(len(full_beh_data))])
midlines_og =  np.array([full_beh_data[i]["midlines"][0:1599] for i in range(len(full_beh_data))])
body_angles_og =  np.array([full_beh_data[i]["body_angles"][0:1599] for i in range(len(full_beh_data))])

velocity = copy.deepcopy(velocity_og)
acceleration = copy.deepcopy(acceleration_og)
head_curvature = copy.deepcopy(head_curvature_og)
worm_curvature = copy.deepcopy(worm_curvature_og)
pumping = copy.deepcopy(pumping_og)
body_angle_mag =copy.deepcopy(body_angle_mag_og)
midlines =copy.deepcopy( midlines_og)
body_angles = copy.deepcopy(body_angles_og)




pdf_save_dir = "/Users/friederikebuck/Downloads/worm notes/rslds_all_dates/"
os.makedirs(pdf_save_dir, exist_ok=True)
directories = glob.glob(json_dir+"*.json") #get filename/paths of all the json files
exp_dates = [directory.split("/")[-1].split(".")[0] for directory in directories] #extract the exp_dates
# exp_date_to_jpeg_dir = {exp_date: exp_to_jpeg_dir(exp_date) for exp_date in exp_dates} ###for not wusing 10sx eesmapled..s hould use confocal..? go from rslds to 20 

   
kappas = np.logspace(3,7,5)
kappas = [kappas[0]]
decrease_kappa_factor = 10
num_ar_iters = 50
num_full_iters = 200
time_bin = 10
prefix = 'my_kappa_scan'
# for kappa in kappas: #kappas[:2]
kappa = kappas[0]
print(f"Fitting model with kappa={kappa}")
model_name = f'{prefix}-{kappa}'
# fig_save_dir = os.path.join(save_dir, model_name)

np.random.seed(0)

foldername = "/Users/friederikebuck/Downloads/worm notes/processed_checkpts/test_rslds_QM_run_061325/model_selection/top_2_combined/"
items = np.array(os.listdir(foldername))
folders = np.array(["." not in directory for directory in items], dtype=bool)
folders = items[folders]

folder = folders[0]


with open(foldername+"/Y.npy", 'rb') as handle:  
    Y = pickle.load(handle)


with open(foldername+folder+"/saved_data/prior.npy", 'rb') as handle:
    global_worm = pickle.load(handle)
    global_worm.D = int(global_worm.D)

with open(foldername+folder+"/saved_data/prior_q.npy", 'rb') as handle:
    q = pickle.load(handle)
    

slds = global_worm

neural_labels = np.load(foldername+"/neurons.npy", allow_pickle=True)


print(np.array(Y).shape)

print(len(neural_labels))



slds = global_worm
# get estimated latents, states
q_x = q.mean_continuous_states
q_z = [slds.most_likely_states(q_x[w], Y[w]) for w in range(len(q_x))]

z = [beh_classification["is_fwd"][0:1599]+ 2*beh_classification["is_rev"][0:1599]+ 4*beh_classification["is_pause"][0:1599]+3*beh_classification["is_turn"][0:1599]-1 for beh_classification in full_beh_classification]
q_z= np.array(q_z)
print(q_z.shape)
z = np.array(z)
print(z.shape)

Worm 0, date: 2022-06-14-01
Worm 1, date: 2023-01-23-08
Worm 2, date: 2022-06-28-07
Worm 3, date: 2022-07-15-12
Worm 4, date: 2023-01-19-01
Worm 5, date: 2022-08-02-01
Worm 6, date: 2022-06-14-07
Worm 7, date: 2022-06-28-01
Worm 8, date: 2023-01-23-15
Worm 9, date: 2022-07-26-01
Worm 10, date: 2023-01-19-15
Worm 11, date: 2023-03-07-01
Worm 12, date: 2022-06-14-13
Worm 13, date: 2023-01-23-21
Worm 14, date: 2023-01-19-22
Worm 15, date: 2023-01-23-01
Worm 16, date: 2023-01-17-01
Worm 17, date: 2022-07-20-01
Worm 18, date: 2023-01-09-28
Worm 19, date: 2023-01-19-08
Worm 20, date: 2022-07-15-06


/Users/friederikebuck/Desktop/MBL/project/WholeBrainImagingAnalysis/collab/beh_classification/behavior_features.py:11: RuntimeWarning: Mean of empty slice
  body_angles = body_angles-np.nanmean(body_angles, axis = 1)[:, None]
/Users/friederikebuck/Desktop/MBL/project/WholeBrainImagingAnalysis/collab/beh_classification/behavior_features.py:15: RuntimeWarning: Mean of empty slice
  centroids = np.nanmean(midlines, axis = 1)


Fitting model with kappa=1000.0
(21, 1599, 138)
138
(21, 1599)
(21, 1599)


In [6]:

motor_state_num_to_name = {
    0:"fwd", 
    1:"rev", 
    2:"turn", 
    3:"pure rev", 
    4:"rev of rev turn", 
    5:"turn of rev turn", 
    6:"pure turn"
}
rslds_state_num_to_name = {
    0:"fwd-like", 
    1:"rev-like", 
    2:"turn-like", 
    3:"pure rev", 
    4:"rev of rev turn", 
    5:"turn of rev turn", 
    6:"pure turn"
}

feature_to_dict = {
    "n body swings":
        {
        # "exp_date_to_feature":  {date: get_body_angle_mag_from_midline(midline)[1][:, 20] for date,midline in exp_date_to_midlines.items() },
        "bins": np.arange(0, 15, 1), 
         "feature_fnc": lambda body_angle_over_event: len(get_zero_crossing_indices(body_angle_over_event)), 
         "ylim": [0, 0.5],
          "ylabel": "P(n body swings)", 
        #   "feature_mat": np.concatenate([get_body_angle_mag_from_midline(midline)[1][:, 20][:, None] for date, midline in exp_date_to_midlines.items()], axis = 1), 
        "feature_mat": body_angles[:,:, 20]
    },
        
    "duration (s)": {
        "exp_date_to_feature": "", 
        "bins": np.linspace(0, 45, 25), 
         "feature_fnc": lambda x: np.nanmax(x), 
         "ylim": [0, 0.5],
          "ylabel": "P(duration)", 
          "feature_mat": "", 
        
    },
        
    "curviness (radians)":
        {
            # "exp_date_to_feature": exp_date_to_body_angle_mag, 
         "bins": np.linspace(0,16, 25), 
         "feature_fnc": lambda x: np.nanmax(x), 
         "ylim": [0, 0.2], 
          "ylabel": "P(curviness)", 
        #   "feature_mat": np.concatenate([exp_date_to_body_angle_mag[date][:, None] for date in exp_dates], axis = 1)
        "feature_mat": body_angle_mag
        } , 
    # "velocity (mm/s)": (exp_date_to_velocity,   np.linspace(0,0.35, 50),  lambda x: np.max(np.abs(x))), 
    "speed (um/s)": 
        {
            # "exp_date_to_feature": exp_date_to_velocity, 
         "bins":  np.linspace(0,350, 25), 
         "feature_fnc": lambda x:np.nanmax(np.abs(x)), 
         "ylim": [0, 0.24], 
         "ylabel": "P(max speed)", 
        #  "feature_mat": np.concatenate([exp_date_to_velocity[date][:, None] for date in exp_dates], axis = 1)
        "feature_mat": velocity
        } , 
    # # "head angle": (exp_date_to_head_angle,  50,  lambda x: np.mean(x)),#np.arange(0, 6, 0.01)), , 
    "acceleration (um/s^2)": 
        {
            # "exp_date_to_feature": exp_date_to_acceleration, 
         "bins":  np.linspace(0,0.015, 25),
         "feature_fnc":  lambda x: np.nanmax(np.abs(x)), 
         "ylim": [0, 0.14] ,
          "ylabel": "P(max acceleration)", 
        #   "feature_mat": np.concatenate([exp_date_to_acceleration[date][:, None] for date in exp_dates], axis = 1). 
         "feature_mat": acceleration
        } , 
    
}


In [7]:
def get_date_to_state_bin(date_to_start_ends, n_frames):
    date_to_state_bin = {}
    all_state_bin =[]
    for date, (starts, ends) in date_to_start_ends.items():
        state_bin = np.zeros(n_frames)
        for start, end in zip(starts, ends):
            state_bin[start:end] = 1
        date_to_state_bin[date] = state_bin
        all_state_bin.append(state_bin)
    return date_to_state_bin, all_state_bin
        

def get_all_states_bin(exp_dates,  z, q_z):
    n_dates = len(exp_dates)
    n_frames = len(z[0])
    all_states = np.zeros((n_dates, n_frames))*np.nan
    pair_to_i = {}
    motor_states = [0,1,2]
    for motor_state in motor_states:#[0,1,2]:
        rslds_state_to_exp_date_to_beh_start_end_is =  get_motor_state_start_end_is_rslds_start_end_specific(motor_state, exp_dates, z, q_z,  rslds_states = [0,1,2])
    
        # motor_state_to_rslds_state_to_exp_date_to_beh_start_end_is[motor_state] = rslds_state_to_exp_date_to_beh_start_end_is
        for i, (pair, exp_date_to_beh_start_end_is) in enumerate(rslds_state_to_exp_date_to_beh_start_end_is.items()):
            pair_to_i[(motor_state, pair)] = i
            for j, (date, start_ends) in enumerate(exp_date_to_beh_start_end_is.items()):
                starts, ends = start_ends
                # print("start_ends", starts, ends )
                for start, end in zip(starts, ends):
                    # print(start)
                    all_states[j, start:end] = i
    return pair_to_i, all_states


def get_transitions(stateseq):
    ''' copied from keypoint moseq github: keypoint-moseq/keypoint_moseq/util.py'''
    transitions = np.nonzero(stateseq[1:] != stateseq[:-1])[0] + 1
    starts = np.insert(transitions, 0, 0)
    ends = np.append(transitions, len(stateseq))
    return  transitions, starts, ends



    
def get_state_start_ends(state, stateseq,  starts= None, ends = None):
    if starts is None: 
        transitions, starts, ends = get_transitions(stateseq)
    # for state1, state2 in state_pairs:
    state_start_is = np.argwhere(state==stateseq[starts[:-1]]).flatten()
    
    state_starts = starts[state_start_is]
    state_ends = ends[state_start_is] ##check 
    
    return state_starts, state_ends
 

In [56]:
def get_motor_state_start_end_is_rslds_start_end_specific_1(motor_states, rslds_states, z, q_z):

    state_to_track_start_end_is = {}
    state_to_lbl = {}
    rslds_pairs = [(x,y) for x in rslds_states for y in rslds_states]
    state = 0
    for motor_state in motor_states:#[0,1,2]:
        for rslds_start_state, rslds_end_state in rslds_pairs: 
            
            state_lbl = (motor_state,(rslds_start_state, rslds_end_state) )
            state_to_lbl[state] = state_lbl
            
            all_tracks = []
            all_starts = []
            all_ends = []
            for track, (z_w, q_z_w) in enumerate(zip( z, q_z)):# (exp_date, behseq) in exp_date_to_beh.items():
                beh_starts, beh_ends = get_state_start_ends(motor_state, z_w,  starts= None, ends = None)
                
                '''get where start in rslds state '''
                rev_is = np.argwhere(
                    np.logical_and(
                    rslds_start_state==q_z_w[beh_starts]
                                        , 
                                        rslds_end_state==q_z_w[beh_ends-1]
                                        
                                        )
                                    ).flatten()
                # total_behs+=rev_is.shape[0]
                starts, ends = beh_starts[rev_is], beh_ends[rev_is]

                all_tracks.append(np.ones(starts.shape[0])*track)
                all_starts.append(starts)
                all_ends.append(ends)
            state_to_track_start_end_is[state] = np.concatenate([
                                                    np.concatenate(all_tracks)[:,None],
                                                    np.concatenate(all_starts)[:,None],
                                                    np.concatenate(all_ends)[:,None]
                                                ], axis = 1).astype('int')
            state+=1

    return state_to_lbl,  state_to_track_start_end_is

def get_features_from_track_start_end_is(feature_to_dict, state_to_track_start_end_is, features = None):
    feature_state_to_feature_vals = {feature: {} for feature in feature_to_dict.keys()}
    # states = state_to_start_end_is.keys()
    if features is None: 
        features = feature_to_dict.keys()
    
    for feature in features: 
        feature_params = feature_to_dict[feature]
        feature_mat = feature_params["feature_mat"]
        feature_func = feature_params["feature_fnc"]
        print("feature", feature)
        if  feature == "duration (s)": 
            for state, track_start_end_is in state_to_track_start_end_is.items():
                feature_state_to_feature_vals["duration (s)"][state] = track_start_end_is[:,2] - track_start_end_is[:, 1]


        for state, track_start_end_is in state_to_track_start_end_is.items():
            feature_vals = []
            
            # print("track_start_end_is", track_start_end_is)
            for track, start, end in track_start_end_is:
                feature_vals.append(feature_func(feature_mat[track, start:end]))

            feature_state_to_feature_vals[feature][state] = np.array(feature_vals)


    return feature_state_to_feature_vals
        
def remove_nans(vals):
    vals = np.array(vals)  
    return vals[~np.isnan(vals)] 

def visualize_transition_matrix(transition_matrix, title = "Transition Matrix",  labels=None):
    """
    Visualizes the transition matrix using a heatmap.

    Parameters:
        transition_matrix (np.ndarray): 2D numpy array representing the transition matrix.
        labels (list or None): List of state labels. If None, indices are used.
    """
    fig, ax = plt.subplots(figsize=(8, 6))
    magma_cmap = plt.cm.magma
    # Create a truncated version of the colormap
    # Use the Normalize function to scale the data between 0 and 0.75
    magma_cmap = mcolors.LinearSegmentedColormap.from_list(
    'truncated_magma', magma_cmap(np.linspace(0.1, 0.75, 256))
    )

    
    
    cax = ax.matshow(transition_matrix, cmap=magma_cmap)
    plt.colorbar(cax, ax=ax)

    # Set labels
    if labels is None:
        labels = [str(i) for i in range(transition_matrix.shape[0])]
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)


    ax.set_title(title, fontsize=14)
    ax.set_xlabel("To State", fontsize=12)
    ax.set_ylabel("From State", fontsize=12)
    ax.tick_params(axis='x', labelsize=20) 
    ax.tick_params(axis='y', labelsize=20) 

    # Add text annotations
    for i in range(transition_matrix.shape[0]):
        for j in range(transition_matrix.shape[1]):
            value = transition_matrix[i, j]
            if value > 0:
                # ax.text(j, i, f"{value:.2f}", ha="center", va="center", color="white" if value > 0.5 else "black")
                # ax.text(j, i, f"{value:.2f}", ha="center", va="center", color= "black")
                ax.text(j, i, f"{value:.2f}", ha="center", va="center", color= "white", fontsize=20 )

    

    return fig, ax

            

In [9]:
from scipy.stats import ks_2samp

def plot_probabilities_hist(np_array, bins, color = "blue", 
                            alpha = 0.7, label ="",
                             fig = None, ax = None, 
                             text ="", x_ratio = 9/12, y_ratio = 10/12, ylim = None, fill = True, 
                             linewidth=2):#fromstackoverflowhttps://stackoverflow.com/questions/3866520/plot-a-histogram-such-that-bar-heights-sum-to-1-probability
    if fig is None: 
        fig, ax = plt.subplots()
    weights = np.ones_like(np_array)/len(np_array)
    if not fill: 
       
        
        ax.hist(np_array, bins = bins, weights= weights, color = color, alpha = alpha, histtype='step',  label = label, fill = fill, linewidth = 2)
    else: 
         ax.hist(np_array, bins = bins, weights= weights, color = color, alpha = alpha, label = label)
    # if len(text)>0:
    #     add_text_to_upper_right_corner(ax, text,  x_ratio = x_ratio, y_ratio = y_ratio)
    # if ylim is None: 
    #     ax.set_ylim([0,1.1])
    return fig, ax


def plot_hist_across_conditins_and_get_KS_stats(cond_to_data,  conditon_to_color,
                                               title = "", 
                                               ylabel = "",
                                               xlabel = "",
                                               stat_color = "dark" ,
                                               bins = None, 
                                            #    ylim = [0,1.1],
                                            ylim = None, 
                                               alpha = 0.7, 
                                            #    linewidth=2, 
                                            fill =  False,
                                               fig = None, 
                                               ax = None , 
                                               print_sig = True,
                                               print_effect_size = False, 
                                               legend = False, condition_to_alpha = None, 
                                               plot_medians = False, 
                                               print_median_diff = False
                                
                                               ):
    
    
        
    if ax is None: 
        fig, ax = plt.subplots()
    medians = []
    for condition, data  in cond_to_data.items():
            
        counts, bin_edges = np.histogram(data, bins=bins, weights=np.ones_like(data)/len(data), density=False)
        cdf = np.cumsum(counts)
        
        # Normalize the CDF to ensure it ends at 1
        cdf = cdf / cdf[-1]

        if condition_to_alpha is None: 
            
            plot_probabilities_hist(data,  
                    bins,
                    conditon_to_color[condition], 
                    alpha = alpha, 
                    label = condition,
            
                    fig = fig, ax = ax, 
                    fill = fill, 
                    # text ="",
                    )
            
                
        else: 
            plot_probabilities_hist(data,  
                    bins,
                    conditon_to_color[condition], 
                    alpha = condition_to_alpha[condition], 
                    label = condition,
            
                    fig = fig, ax = ax, 
                    fill = fill, 
                    # text ="", 
                    )
        if plot_medians: 
            median = np.median(data)
            medians.append(median)
        
            ax.axvline(median, linestyle = "--", color = conditon_to_color[condition], alpha = alpha)
    if legend: 
        ax.legend()
    if ylim is not None: 
        ax.set_ylim(ylim)
    ax.set_title(title,  fontsize=19)
    ax.set_ylabel(ylabel, fontsize=19)
    ax.set_xlabel(xlabel, fontsize=19)


    ax.tick_params(axis='x', labelsize=14)  
    ax.tick_params(axis='y', labelsize=14)  
    
    # ax.spines['top'].set_visible(False)   # Hide top spine
    # ax.spines['right'].set_visible(False)  # Hide right spine

    # ax.spines['left'].set_linewidth(2)   # Left spine bold
    # ax.spines['bottom'].set_linewidth(2)  # Bottom spine bold
    # fig.set_size_inches(4, 5)

    # Compute KS Test if reference_data is provided
    ks_stat, p_value = None, None
    conditions = list(cond_to_data.keys())

    ks_stat, p_value = ks_2samp(cond_to_data[conditions[0]], cond_to_data[conditions[1]])  # 2-sample KS test
    
    # Determine significance stars
    if p_value < 0.001:
        stars = "***"
    elif p_value < 0.01:
        stars = "**"
    elif p_value < 0.05:
        stars = "*"
        
    else:
        stars = "ns"  # Not significant
    if print_effect_size and stars!= "ns": 
        stars = stars+ " " + str(np.round(ks_stat,2))
    if print_median_diff and len(medians)==2: 
        stars = stars+ " " + str(np.abs(np.round(np.diff(np.array(medians)),2))[0])
        ax.text(0.95, 0.95, stars, transform=ax.transAxes, fontsize=25, color=stat_color, ha='right', va='top' , fontweight='bold')
    
    if print_sig: 
    # Add stars to the plot in the lower-right corner
        ax.text(0.95, 0.95, stars, transform=ax.transAxes, fontsize=25, color=stat_color, ha='right', va='top' , fontweight='bold')

    
    # Optionally print results
    # print(f"KS Statistic: {ks_stat}, p-value: {p_value}")
    fig.tight_layout()
    return fig, ax 

In [54]:
def get_rslds_pair_counts(state_to_track_start_end_is, state_to_label, motor_states, rslds_states, plot= True,rslds_state_num_to_name = None ):
    label_to_state = {val:key for key, val in state_to_label.items()}
    rslds_pairs = [(x,y) for x in rslds_states for y in rslds_states]
    motor_state_to_counts ={}
    for motor_state in motor_states:#[0,1,2]:
        counts = np.zeros((len(rslds_states),  len(rslds_states)))
        for start, end in rslds_pairs:
            state = label_to_state[(motor_state, (start, end))]
            counts[start, end] = state_to_track_start_end_is[state].shape[0]
        motor_state_to_counts[motor_state] = counts
        if plot:
            visualize_transition_matrix(counts, title = f"start, end rslds state of each {motor_state}",   labels= [rslds_state_num_to_name[i] for i in range(3)] )    
    return motor_state_to_counts

def plot_feature_hists_motor_states(feature_state_to_feature_vals,feature_to_dict, rslds_state_to_color,  state_to_label, motor_states, rslds_states, features = None):
    label_to_state = {val:key for key, val in state_to_label.items()}
    
    # rslds_pairs = [(x,y) for x in rslds_states for y in rslds_states]
    if features is None: 
        features = feature_state_to_feature_vals.keys()
        
    for feature in features: 
        state_to_feature_vals  = feature_state_to_feature_vals[feature]
        
        for motor_state in motor_states:#[0,1,2]:
            for rslds_start_state in rslds_states:
                fig1, ax1 = plt.subplots()
                for rslds_end_state in rslds_states:
                    state = label_to_state[(motor_state, (rslds_start_state, rslds_end_state ))]
                    feature_vals = state_to_feature_vals[state]

                    ax1.hist(np.array(feature_vals),
                                            # bins= feature_to_dict[feature]["bins"], 
                                            bins = feature_to_dict[feature]["bins"], # np.arange(0, 15, 1)
                                            color = rslds_state_to_color[rslds_end_state], 
                                            alpha = 0.3)
        
                    ax1.set_title(f"{feature}; \nmotor state: {motor_state_num_to_name[motor_state]};  \n rslds start:{rslds_state_num_to_name[rslds_start_state]}")
            
            
                rslds_state_to_feature = {rslds_end_state: remove_nans( state_to_feature_vals[label_to_state[(motor_state, (rslds_start_state, rslds_end_state ))]]) for  rslds_end_state in rslds_states}
                fig1, ax1 = plt.subplots()
                if np.all(np.array([feature_val.size >0 for feature_val in rslds_state_to_feature.values()])):
                    plot_hist_across_conditins_and_get_KS_stats(rslds_state_to_feature,  rslds_state_to_color,
                                                            title = f"{feature} \n motor state: {motor_state_num_to_name[motor_state]}; \n rslds start:{rslds_state_num_to_name[rslds_start_state]}", 
                                                            ylabel = feature_to_dict[feature]["ylabel"],
                                                            xlabel = f"{feature}",
                                                            stat_color = "tomato" ,
                                                            bins = feature_to_dict[feature]["bins"], # np.arange(0, 15, 1)
                                                            ylim = feature_to_dict[feature]["ylim"], 
                                                            alpha = 0.3, 
                                                        #    linewidth=2, 
                                                            fill =  True,
                                                            fig = fig1, 
                                                            ax = ax1 , 
                                                            print_sig = False,
                                                            print_effect_size = False, 
                                                            legend = False,
                                                            plot_medians = True
                                                            )
# rslds_states = [0,1,2]
# motor_states = [0,1,2]
# state_to_label,  state_to_track_start_end_is = get_motor_state_start_end_is_rslds_start_end_specific_1(motor_states, rslds_states, z, q_z)
# feature_state_to_feature_vals = get_features_from_track_start_end_is(feature_to_dict, state_to_track_start_end_is )
# get_rslds_pair_counts(state_to_track_start_end_is, state_to_label, motor_states, rslds_states, plot= True, rslds_state_num_to_name = rslds_state_num_to_name )
# plot_feature_hists_motor_states(feature_state_to_feature_vals,feature_to_dict, rslds_state_to_color,  state_to_label, motor_states, rslds_states)



motor_states = [3,4,5,6]
rslds_states = [0,1,2]

# reor_mat =  get_reor_mat(z)
# state_to_label,  state_to_track_start_end_is = get_motor_state_start_end_is_rslds_start_end_specific_1(motor_states, rslds_states, reor_mat, q_z)
# feature_state_to_feature_vals = get_features_from_track_start_end_is(feature_to_dict, state_to_track_start_end_is )
# get_rslds_pair_counts(state_to_track_start_end_is, state_to_label, motor_states, rslds_states, plot= True, rslds_state_num_to_name = rslds_state_num_to_name )
# plot_feature_hists_motor_states(feature_state_to_feature_vals,feature_to_dict, rslds_state_to_color,  state_to_label, motor_states, rslds_states)



In [60]:
''' plot neural activity'''

neural_feature_to_dict = {
    "neural_activity_mean":
        {
        # "exp_date_to_feature":  {date: get_body_angle_mag_from_midline(midline)[1][:, 20] for date,midline in exp_date_to_midlines.items() },
        # "bins": np.arange(0, 15, 1), 
         "feature_fnc": lambda neuralseq: np.nanmean(neuralseq, axis = 0), 
         "ylim": [0, 0.5],
          "ylabel": "mean", 
              "feature_mat": np.array(Y)
    },
            "neural_activity_min":
        {
        # "exp_date_to_feature":  {date: get_body_angle_mag_from_midline(midline)[1][:, 20] for date,midline in exp_date_to_midlines.items() },
        # "bins": np.arange(0, 15, 1), 
         "feature_fnc": lambda neuralseq: np.nanmin(neuralseq, axis = 0), 
         "ylim": [0, 0.5],
          "ylabel": "mean", 
              "feature_mat": np.array(Y)
    },
                    "neural_activity_max":
        {
        # "exp_date_to_feature":  {date: get_body_angle_mag_from_midline(midline)[1][:, 20] for date,midline in exp_date_to_midlines.items() },
        # "bins": np.arange(0, 15, 1), 
         "feature_fnc": lambda neuralseq: np.nanmax(neuralseq, axis = 0), 
         "ylim": [0, 0.5],
          "ylabel": "mean", 
              "feature_mat": np.array(Y)
    },
        
    
}

def get_pair_wise_avgs(pair1_vals, pair2_vals, max_comparisons = 1000, random_state=None):
    rng = np.random.default_rng(random_state)
    # n_samples = min(len(pair1_vals), len(pair2_vals))
    # n_samples = min(n_samples, max_comparisons)
    n_samples = 1000
    idx1 = rng.integers(0, len(pair1_vals), size=n_samples)
    idx2 = rng.integers(0, len(pair2_vals), size=n_samples)

    diffs = pair1_vals[idx1] - pair2_vals[idx2]

    return diffs


def plot_neural_activity_heatmap(columns, neural_labels_all, col_titles, neural_labels_to_plot = None, border_neurons = None):
    
    if neural_labels_to_plot is None: 
        neural_labels_to_plot  = neural_labels_all
    neural_labels_to_plot = np.array(neural_labels_to_plot)
    indices = [np.where(neural_labels_all == val)[0][0] for val in neural_labels_to_plot]
    
    
    
    columns = np.array(columns)
    n_rows, n_cols = columns.shape
    fig, axs = plt.subplots( 1, n_cols ,figsize=(12, 30))
    
    
    for i in range(n_cols):
        col, title = columns[:, i], col_titles[i]
        axs[i].imshow(col[indices][:,None], aspect="auto", cmap="coolwarm", vmin = -2, vmax=2)
        axs[i].set_title(title)
    

    ax = axs[0]
    ax.set_yticks(np.arange(len(neural_labels_to_plot)), labels=neural_labels_to_plot, #rotation = 45, 
                  fontsize=10)
    

    if border_neurons is not None:
            for neuron in border_neurons: 
                i = np.argwhere(neural_labels_to_plot == neuron)[0]
                for ax in axs: 
                    ax.axhline(i-0.5, color = "black")
    


def plot_neural_activity(neural_feature_to_dict,state_to_track_start_end_is, label_to_state):
    # pairs = list(pair_to_vals_dict.keys())
    state_to_neural_activity_means = get_features_from_track_start_end_is(neural_feature_to_dict, state_to_track_start_end_is, features = ["neural_activity_mean"] )["neural_activity_mean"]
    pairs = [(x,y) for x in rslds_states for y in rslds_states]
    for motor_state in motor_states:
    
            
        

        for i in range(len(pairs)):
            
            pair1 = pairs[i]
            state1 = label_to_state[(motor_state, pair1)]
            pair1_vals = np.array(state_to_neural_activity_means[state1])
            n_pair1_instances = len(pair1_vals)
            if n_pair1_instances<15:
                continue
            print(pair1 , n_pair1_instances)
            pair_specific_mean_diffs = []

            col_titles = []

            for j in range(len(pairs)):
                # ax = axs[k]
                if i ==j:
                    continue
                pair2 = pairs[j]
                state2 = label_to_state[(motor_state, pair2)]
                pair2_vals = np.array(state_to_neural_activity_means[state2])
                n_pair1_instances = len(pair1_vals)
                n_pair2_instances = len(pair2_vals)
                if n_pair2_instances<15:
                    print("skipping, ", pair2, n_pair2_instances)
                    continue
                diffs  = get_pair_wise_avgs(pair1_vals, pair2_vals, max_comparisons = 1000, random_state=None)
                mean_diffs = np.nanmean(diffs, axis = 0 )[:, None]
                ##plot heat map of diffs with neaul activity as labels 
            
                # im = ax.imshow(mean_diffs, aspect="auto", cmap="coolwarm", vmin = -0.3, vmax=0.3)
                pair_specific_mean_diffs.append(mean_diffs)
                col_titles.append(f"{pair1}, {pair2}, {motor_state_num_to_name[motor_state]}")

            pair_specific_mean_diffs = np.concatenate(pair_specific_mean_diffs, axis = 1)
            plot_neural_activity_heatmap(pair_specific_mean_diffs, neural_labels, col_titles, neural_labels_to_plot = neural_labels_cropped, border_neurons = ['dF - AVE'
        , "dF - AVE", "F - OLL"])

import os
contributor_thres = 0.05
foldername = "/Users/friederikebuck/Downloads/worm notes/processed_checkpts/test_rslds_QM_run_061325/model_selection/top_2_combined/"
with open(os.path.join(foldername, f'high_contributing_neurons_rslds_{contributor_thres}.txt')) as f:
    neural_labels_cropped = [line.strip() for line in f]
rslds_states = [0,1,2]
motor_states = [0,1,2]
state_to_label,  state_to_track_start_end_is = get_motor_state_start_end_is_rslds_start_end_specific_1(motor_states, rslds_states, z, q_z)
label_to_state = {val:key for key, val in state_to_label.items()}
plot_neural_activity(neural_feature_to_dict, state_to_track_start_end_is, label_to_state)


# feature_state_to_feature_vals = get_features_from_track_start_end_is(feature_to_dict, state_to_track_start_end_is )
# get_rslds_pair_counts(state_to_track_start_end_is, state_to_label, motor_states, rslds_states, plot= True, rslds_state_num_to_name = rslds_state_num_to_name )
# plot_feature_hists_motor_states(feature_state_to_feature_vals,feature_to_dict, rslds_state_to_color,  state_to_label, motor_states, rslds_states)



plt.show()

feature neural_activity_mean


KeyError: 0

In [58]:
state_to_track_start_end_is

{0: array([[   0,  308,  309],
        [   0,  615,  620],
        [   0, 1076, 1087],
        [   0, 1148, 1151],
        [   0, 1264, 1278],
        [   0, 1577, 1581],
        [   1,  135,  157],
        [   1,  578,  647],
        [   1,  911,  923],
        [   1, 1064, 1068],
        [   2,  192,  234],
        [   2,  246,  256],
        [   2,  355,  373],
        [   2,  376,  385],
        [   2,  812,  859],
        [   2, 1328, 1352],
        [   2, 1503, 1534],
        [   2, 1561, 1596],
        [   3,  219,  230],
        [   3,  232,  234],
        [   3, 1514, 1551],
        [   4,  303,  314],
        [   4,  317,  326],
        [   4,  409,  414],
        [   4,  419,  425],
        [   4,  512,  526],
        [   4,  528,  532],
        [   4,  535,  542],
        [   4,  576,  583],
        [   4, 1310, 1322],
        [   4, 1373, 1392],
        [   5,  331,  361],
        [   6,    4,    8],
        [   6,  211,  236],
        [   6,  399,  417],
        [   6,  4

In [ ]:
'''get vids '''

(21, 1599)

In [ ]:
'''seperate if high vs low neural activity '''

dict_items([(0, (0, (0, 0))), (1, (0, (0, 1))), (2, (0, (0, 2))), (3, (0, (1, 0))), (4, (0, (1, 1))), (5, (0, (1, 2))), (6, (0, (2, 0))), (7, (0, (2, 1))), (8, (0, (2, 2))), (9, (1, (0, 0))), (10, (1, (0, 1))), (11, (1, (0, 2))), (12, (1, (1, 0))), (13, (1, (1, 1))), (14, (1, (1, 2))), (15, (1, (2, 0))), (16, (1, (2, 1))), (17, (1, (2, 2))), (18, (2, (0, 0))), (19, (2, (0, 1))), (20, (2, (0, 2))), (21, (2, (1, 0))), (22, (2, (1, 1))), (23, (2, (1, 2))), (24, (2, (2, 0))), (25, (2, (2, 1))), (26, (2, (2, 2)))])